# Capstone: Recommender System
## 4. Collaborative Recommender System

In [18]:
import pandas as pd
import numpy as np
from surprise import SVD, KNNWithMeans, SVDpp, KNNBasic
from surprise import accuracy
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

In [374]:
pip install surprise

    100% |████████████████████████████████| 3.3MB 3.6MB/s 
  Stored in directory: /Users/bestmacbook/Library/Caches/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [4]:
review = pd.read_csv('../datasets/review_clean.csv')

In [5]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137685 entries, 0 to 137684
Data columns (total 12 columns):
Unnamed: 0      137685 non-null int64
full_name       137685 non-null object
Category        137685 non-null object
brand           137685 non-null object
name            137685 non-null object
user            137685 non-null object
stars           137685 non-null int64
short_review    107655 non-null object
long_review     137685 non-null object
helpfulness     137685 non-null int64
time            137253 non-null object
id              137685 non-null int64
dtypes: int64(4), object(8)
memory usage: 12.6+ MB


In [7]:
review.drop(columns = ['Unnamed: 0'], inplace = True)

In [8]:
#constract a users vs product matrix, it contains rating products from users
up = pd.pivot_table(review, index='user', columns='id', values='stars').fillna(0)

In [9]:
up

id,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,278,279,280
user,,,,,,,,,,,,,,,,,,,,,
00022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001000101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001122334455667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00450045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007Barbie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007CatWomen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007DCUL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007gal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00Greenpea00,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#find users who only purchase one product
l = []
for i in up.index:
    s = sum([1 for i in up.loc[i] if i != 0.0])
    if s < 2:
        l.append(i)

In [11]:
len(l)

93326

In [12]:
#drop
up.drop(index = l, inplace = True)

In [13]:
len(up)

17278

In [14]:
#find one time purchased users index number in review dataset
indexlist = []
for i in l:
    n = review[review.user == i].index[0]
    indexlist.append(n)

In [15]:
#drop those users for review dataframe and create a new dataset review1
review1 = review[['user', 'id', 'stars']]
review1 = review1.drop(index = indexlist)


In [16]:
len(review1)

44359

## Model Evaluation with package surprise

In [19]:
#data preparetions
# set rating_scale param 
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(review1[['user', 'id', 'stars']], reader)

#train test split
trainset, testset = train_test_split(data, test_size = 0.25)



## SVD

In [20]:
#Grid Search for the best parameters
param_grid = {'n_factors':[40, 60], 
              'n_epochs': [20, 40, 60],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.1, 0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)
# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.1436829032811826
{'n_factors': 40, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.4}


In [21]:
#run the SVD model
algo = gs.best_estimator['rmse']
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred)

RMSE: 1.1510


1.1510320199965756

## SVD++

In [22]:
#Grid Search for the best parameters
param_grid = {'n_factors':[40, 60], 'n_epochs': [20, 40], 'lr_all': [0.005, 0.007],
              'reg_all': [0.2, 0.4, 0.6]}
gs1 = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

gs1.fit(data)

# best RMSE score
print(gs1.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs1.best_params['rmse'])

1.1450478734624998
{'n_factors': 40, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.2}


In [23]:
algo = gs1.best_estimator['rmse']
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred)

RMSE: 1.1507


1.1507340319684727

## KNN

In [24]:
algo = KNNBasic(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

# run the trained model against the testset
test_pred = algo.test(testset)

# get RMSE
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.2816


1.2815959846910523

By comparing the RMSE values of three models, SVD and SVD++ give us better result.

And then, we will construct a matix with SVD.


In [41]:
up_matrix = up.values

In [26]:
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(up_matrix , k = 40)

In [27]:
sigma = np.diag(sigma)
sigma.shape

(40, 40)

After the factorization, we try to to reconstruct the original matrix by multiplying its factors. The resulting matrix is not sparse any more. It was generated predictions for items the user have not yet interaction, which we will exploit for recommendations.

In [38]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings.shape

(17278, 281)

In [40]:
#Converting the reconstructed matrix back to a Pandas dataframe
item_based = pd.DataFrame(all_user_predicted_ratings, columns = up.columns, index=list(up.index)).transpose()
item_based.shape


(281, 17278)

In [30]:
sim_matrix = cosine_similarity(item_based)
collaborative_sim = pd.DataFrame(sim_matrix, columns=item_based.index, index=item_based.index)

In [31]:
collaborative_sim.head()

id,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,278,279,280
id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.007424,0.014467,0.212735,0.028928,0.001812,0.008993,0.038045,-0.013437,0.016253,...,0.023221,0.012636,0.108990,-0.007754,0.261532,0.235143,-0.032235,0.081715,0.198258,-0.028750
1,0.007424,1.000000,0.059456,0.150035,-0.013574,0.034248,0.020623,0.033282,0.815632,0.032876,...,-0.008224,0.315635,0.091877,0.357517,-0.008500,0.009846,-0.022590,0.375153,-0.016026,0.495060
2,0.014467,0.059456,1.000000,0.074073,0.009476,-0.003730,0.019439,0.063996,0.171413,0.025225,...,-0.012548,0.389529,0.001757,0.000118,-0.000867,0.185602,-0.003201,0.354348,-0.021057,0.269540
3,0.212735,0.150035,0.074073,1.000000,0.121596,0.370508,0.219642,0.133643,0.417675,0.004828,...,0.039302,0.383658,0.222548,0.052362,0.153487,0.162229,0.321975,0.297825,0.304418,0.296123
4,0.028928,-0.013574,0.009476,0.121596,1.000000,0.246030,0.023239,0.038490,-0.022507,0.011908,...,0.002388,0.001536,0.574438,-0.003643,0.085566,0.006053,-0.036071,0.002719,0.169005,-0.018230


In [553]:
collaborative_sim.to_csv('../datasets/collaborative_sim.csv')